In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

In [2]:
SU = pd.read_csv("../tables/Survey Units.txt", header=None)
exSU = pd.read_excel('../DataFiles/Survey Units.xlsx')

In [3]:
SU.columns = exSU.columns

In [4]:
su = SU[['SR', 'Begin ARM', 'End ARM']]
su.columns = ['SR', 'BA', 'EA']

In [5]:
su.head()

,SR,BA,EA
0,2,312.04,312.14
1,2,171.15,171.25
2,2,269.53,269.63
3,2,22.12,22.23
4,2,115.81,115.91


In [6]:
sr = np.unique(np.array(su.SR))

In [7]:
list(su[su.SR == 2].min())[1]

0.0

In [8]:
df = pd.DataFrame(sr)

In [9]:
df.columns = ['SR']

In [10]:
def addMin(row):
    sr = row['SR']
    return list(su[su.SR == sr].min())[1]
def addMax(row):
    sr = row['SR']
    return list(su[su.SR == sr].max())[2]

In [11]:
df.loc[:, 'Begins'] = df.apply(addMin, axis = 1)
df.loc[:, 'Ends'] = df.apply(addMax, axis = 1)

In [12]:
df.head()

,SR,Begins,Ends
0,2,0,326.34
1,3,0,59.81
2,4,0,62.27
3,5,0,276.62
4,6,0,51.37


In [13]:
d1 = pd.DataFrame(columns=('SR', 'bs', 'es'))

In [14]:
for i in range(5):
    d1.loc[i] = [df.loc[i, 'SR'], list(df[df.SR ==(i+2)].Begins)[0], df.loc[i, 'Begins']+0.1]

In [15]:
df[df.SR ==3].Begins

1    0
Name: Begins, dtype: float64

In [16]:
d1

,SR,bs,es
0,2,0,0.1
1,3,0,0.1
2,4,0,0.1
3,5,0,0.1
4,6,0,0.1


In [17]:
db = pd.DataFrame(columns=('SR', 'bs', 'es'))
for road in np.array(df.SR):
    d1 =  pd.DataFrame(columns=('SR', 'bs', 'es'))
    ns = int(df[df.SR == road].Ends/0.1) + 1
    for i in range(ns):
        if i == 0:
            d1.loc[i] = [road, list(df[df.SR == road].Begins)[0]*10, list(df[df.SR == road].Begins)[0]*10 + 1]
        else:
            d1.loc[i] = [road, d1.loc[(i-1), 'es'] , d1.loc[(i-1), 'es'] + 1]
    db = pd.concat([db,d1])

In [18]:
print np.sum(np.array(df[['Ends']])*10)
print len(db)

72948.8
73046


In [19]:
db.loc[:, 'UID'] = np.arange(len(db))+1

In [20]:
db.head()

,SR,bs,es,UID
0,2,0,1,1
1,2,1,2,2
2,2,2,3,3
3,2,3,4,4
4,2,4,5,5


In [21]:
db.to_csv('../FinalDatasets/sid.csv')

In [22]:
def getUID(row):
    rd = row['SR']
    bg = row['bs']
    end = row['es']
    res = db[((db.SR==rd) & ((db.bs == bg) & (db.es == end)))].UID
    if list(res) == []:
        return None
    else:
        return list(res)[0]

In [23]:
su.head()

,SR,BA,EA
0,2,312.04,312.14
1,2,171.15,171.25
2,2,269.53,269.63
3,2,22.12,22.23
4,2,115.81,115.91


In [24]:
def roundme(n):
    return int(n*10)

In [25]:
su.loc[:, 'bs'] = su.loc[:, 'BA'].apply(roundme)
su.loc[:, 'es'] = su.loc[:, 'EA'].apply(roundme)

/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
su.head(10)

,SR,BA,EA,bs,es
0,2,312.04,312.14,3120,3121
1,2,171.15,171.25,1711,1712
2,2,269.53,269.63,2695,2696
3,2,22.12,22.23,221,222
4,2,115.81,115.91,1158,1159
5,2,263.94,264.04,2639,2640
6,2,58.81,58.85,588,588
7,2,168.75,168.85,1687,1688
8,2,305.11,305.21,3051,3052
9,2,211.71,211.81,2117,2118


In [27]:
su.loc[:, 'UID'] = su.apply(getUID, axis = 1)

In [28]:
print len(su[np.isnan(su.UID)])
print len(su)

4412
91022


In [29]:
su_wid = su[~np.isnan(su.UID)]
su2 = su[np.isnan(su.UID)].sort(columns = ['SR', 'BA', 'EA'])

In [30]:
print len(su_wid)
print len(su2)

86610
4412


In [31]:
# su2.head()

In [32]:
su_ids = np.array(su_wid.UID)

In [33]:
# (su2.loc[4064, 'EA'] - su2.loc[4064, 'BA'] > 0.05)  & (su2.loc[4064, 'EA'] - su2.loc[4064, 'BA'] < 0.11)

In [34]:
def fillUID(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    begin = row['bs']
    if (end - bg > 0.05) & (end - bg < 0.15):
        res = db[((db.SR==rd) & (db.bs == begin))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in su_ids: #  change su_ids to a list and just append x to it if not found in ids.
                return x
            return None

In [35]:
su2.loc[:, 'UID'] = su2.apply(fillUID, axis = 1)

In [36]:
len(su2[np.isnan(su2.UID)])

4218

In [37]:
su_w2 = su2[~np.isnan(su2.UID)]
su3 = su2[np.isnan(su2.UID)]

In [38]:
su_ids = np.concatenate((su_ids,np.array(su_w2.UID)),axis=0)

In [39]:
def fillUID2(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    begin = row['bs']
    if (end - bg > 0.01):
        res = db[((db.SR==rd) & (db.bs == int((bg*10 + end*10)/2)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in su_ids:
                return x
            return None

In [40]:
su3.loc[:, 'UID'] = su3.apply(fillUID2, axis = 1)

In [41]:
temp = su3.dropna(subset = ['UID'])
print len(temp)

46


In [42]:
su_w3 = su3[~np.isnan(su3.UID)]
su4 = su3[np.isnan(su3.UID)]

In [43]:
su_ids = np.concatenate((su_ids,np.array(su_w3.UID)),axis=0)

In [44]:
print len(np.unique(su_ids))
print len(db)
print len(su)

69434
73046
91022


In [45]:
su_final = pd.concat([su_wid, su_w2, su_w3])

In [46]:
su_ds = su_final.drop_duplicates(subset = ['UID'])

In [47]:
print len(su_ds)

69434


In [48]:
SU.head()

,SR,RRT,RRQ,Begin ARM,End ARM,Begin SRMP,End SRMP,Direction To Inventory,Original ID,Due Year,...,Is Near Tunnel,Is Near RR Crossing,Lane Surface Type,Lane Miles,Region,NHS,Federal Functional Class,Avg Construction Date,Avg Major Rehab Date,Int ID
0,2,NaN,NaN,312.04,312.14,320.14,320.24,I,00010e96-fe8c-4c32-a01c-b12814f61879,2018,...,0,0,BST,0.20,Eastern,1,Rural Other Freeway/Expressway,10/1/1970 0:00:00,9/29/2011 0:00:00,1
1,2,NaN,NaN,171.15,171.25,179.20,179.30,B,000a0e11-6103-4075-b19e-7d6e3adff0ce,2014,...,0,0,BST,0.20,North Central,1,Rural Other Principal Arterial,1/1/1954 0:00:00,10/30/2007 0:00:00,2
2,2,NaN,NaN,269.53,269.63,277.61,277.71,I,0018349a-99e4-4bde-a89b-b2f28fa05e82,2024,...,0,0,ACP,0.20,Eastern,1,Urban Other Principal Arterial,1/1/1929 0:00:00,6/4/2014 0:00:00,3
3,2,NaN,NaN,22.12,22.23,22.04,22.15,B,0038b2b9-8cd1-441f-81ae-e2675d09198c,2016,...,0,0,ACP,0.22,Northwest,1,Urban Other Principal Arterial,5/1/1941 0:00:00,6/30/2000 0:00:00,4
4,2,NaN,NaN,115.81,115.91,115.75,115.85,D,003a4aff-0bfd-4e9d-bff4-1adaecfbb025,2013,...,0,0,ACP,0.20,North Central,1,Urban Other Freeway/Expressway,1/1/1953 0:00:00,8/3/2006 0:00:00,5


In [49]:
SU_FINAL = pd.merge(su_ds, SU, how = 'left', left_on = ['SR', 'BA', 'EA'], right_on = ['SR', 'Begin ARM', 'End ARM'])

In [50]:
SU2 = SU_FINAL.dropna(subset = ['Lane Miles'])

In [51]:
SU_FINAL.to_csv('../FinalDatasets/SurveyUnits.csv')

In [52]:
print len(SU2), len(SU_FINAL)

80755 80755


# Contracts dataset

In [53]:
excd= pd.read_excel("../DataFiles/Contracts.xlsx")

In [54]:
cd= pd.read_csv("../tables/Contracts.txt", header = None)
cd.columns = excd.columns

In [55]:
cds = cd[['SR', 'Begin ARM', 'End ARM']]
cds.columns = ['SR', 'BA', 'EA']
cds.loc[:, 'bs'] = cds.loc[:, 'BA'].apply(roundme)
cds.loc[:, 'es'] = cds.loc[:, 'EA'].apply(roundme)
# cdP = cds[~np.isnan(cds.UID)]

In [56]:
cds.loc[:, 'UID'] = cds.apply(getUID, axis = 1)

In [57]:
cds_wid = cds[~np.isnan(cds.UID)]
print len(cds_wid)
cds2 = cds[np.isnan(cds.UID)].sort(columns = ['SR', 'BA', 'EA'])
print len(cds2)

10491
43705


In [58]:
cds_ids = list(np.array(cds_wid.UID))

In [59]:
def fillUID(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    begin = row['bs']
    if (end - bg < 0.15):
        res = db[((db.SR==rd) & (db.bs == begin))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in cds_ids: #  change su_ids to a list and just append x to it if not found in ids.
                cds_ids.append(x)
                return x
            return None

In [60]:
cds2.loc[:, 'UID'] = cds2.apply(fillUID, axis = 1)

In [61]:
cds_w2 = cds2[~np.isnan(cds2.UID)]
print len(cds_w2)
cds3 = cds2[np.isnan(cds2.UID)]
print len(cds3)

1961
41744


In [62]:
print len(cds_ids) - len(cds_wid)

1961


In [63]:
def fillUID2(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    begin = row['bs']
    if (end - bg > 0.15):
        res = db[((db.SR==rd) & (db.bs == int((bg*10 + end*10)/2)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in cds_ids:
                cds_ids.append(x)
                return x
            return None

In [64]:
cds3.loc[:, 'UID'] = cds3.apply(fillUID2, axis = 1)

In [65]:
cds_w3 = cds3[~np.isnan(cds3.UID)]
print len(cds_w3)
cds4 = cds3[np.isnan(cds3.UID)]
print len(cds4)

10944
30800


In [66]:
print len(np.unique(cds_ids))
print len(db)
print len(cds)

16351
73046
54196


In [67]:
def fillUID3(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    endARM = row['es']
    if (end - bg > 0.05) & (end - bg < 0.5):
        res = db[((db.SR==rd) & (db.es == endARM))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in cds_ids:
                cds_ids.append(x)
                return x
            return None

In [68]:
cds4.loc[:, 'UID'] = cds4.apply(fillUID3, axis = 1)

In [69]:
cds_w4 = cds4[~np.isnan(cds4.UID)]
print len(cds_w4)
cds5 = cds4[np.isnan(cds4.UID)]
print len(cds5)

949
29851


In [70]:
def fillUID4(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
#     endARM = row['es']
    if (end - bg > 0.25):
        res = db[((db.SR==rd) & (db.bs == int((bg*10 + end*10)/3)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in cds_ids:
                cds_ids.append(x)
                return x
            return None
def fillUID5(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
#     endARM = row['es']
    if (end - bg > 0.25):
        res = db[((db.SR==rd) & (db.es == int((bg*10 + end*10)/3)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in cds_ids:
                cds_ids.append(x)
                return x
            return None

In [71]:
cds5.loc[:, 'UID'] = cds5.apply(fillUID4, axis = 1)

In [72]:
cds_w5 = cds5[~np.isnan(cds5.UID)]
print len(cds_w5)
cds6 = cds5[np.isnan(cds5.UID)]
print len(cds6)

4023
25828


In [73]:
cds6.loc[:, 'UID'] = cds6.apply(fillUID5, axis = 1)

In [74]:
cds_w6 = cds6[~np.isnan(cds6.UID)]
print len(cds_w6)
cds7 = cds6[np.isnan(cds6.UID)]
print len(cds7)

2271
23557


In [75]:
cds_all = pd.concat([cds_wid, cds_w2, cds_w3, cds_w4, cds_w5, cds_w6])

In [76]:
cds_all = cds_all.drop_duplicates(subset = ['UID'])

In [77]:
cds_final = pd.merge(cds_all, cd, how = 'left', left_on = ['SR', 'BA', 'EA'], right_on = ['SR', 'Begin ARM', 'End ARM'])
cds_FINAL = cds_final.dropna(subset = ['Surface Thickness'])
CDS = cds_FINAL.drop_duplicates(subset = ['SR', 'Begin ARM', 'End ARM', 'Construction End Date'])

In [78]:
print len(np.unique(cds_ids)), len(CDS), len(cd)

23594 27224 54196


In [79]:
CDS[['SR', 'Begin ARM', 'End ARM', 'UID', 'Construction End Date', 'ID']].head()

,SR,Begin ARM,End ARM,UID,Construction End Date,ID
0,2,0.00,0.13,1,4/29/1994 0:00:00,206e5b45-4e3b-4e3f-9a86-d284970ae774
1,2,0.00,0.13,1,10/31/1997 0:00:00,32079337-b228-401a-9503-e0fe9fc52092
3,2,0.00,0.13,1,9/9/9999 0:00:00,73d8794a-336c-449c-bd61-358b09b3bcad
4,2,0.00,0.13,1,7/11/2008 0:00:00,aed59db0-34d4-4d80-97ae-63b50c7b1cd2
5,2,1.49,1.55,15,8/31/1999 0:00:00,6ffa8ecf-d4cd-40f2-aaf7-2857793a14f8


In [80]:
CDS.to_csv('../FInalDatasets/Contracts.csv')

# Contracts History

In [81]:
exCH= pd.read_excel("../DataFiles/Contract History.xlsx")
CH = pd.read_csv('../tables/Contract History.txt', header = None)

In [82]:
CH.columns = exCH.columns
ch = CH[['SR', 'Begin ARM', 'End ARM']]
ch.columns = ['SR', 'BA', 'EA']
ch.loc[:, 'bs'] = ch.loc[:, 'BA'].apply(roundme)
ch.loc[:, 'es'] = ch.loc[:, 'EA'].apply(roundme)
ch.loc[:, 'UID'] = ch.apply(getUID, axis = 1)

In [83]:
ch_wid = ch[~np.isnan(ch.UID)]
print len(ch_wid)
ch2 = ch[np.isnan(ch.UID)].sort(columns = ['SR', 'BA', 'EA'])
print len(ch2)

8354
20234


In [84]:
ch_ids = list(np.unique(np.array(ch_wid.UID)))
print len(ch_ids)

5089


In [85]:
def fillUID(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    begin = row['bs']
    if (end - bg < 0.15):
        res = db[((db.SR==rd) & (db.bs == begin))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in ch_ids: #  change su_ids to a list and just append x to it if not found in ids.
                ch_ids.append(x)
                return x
            return None
    else:
        return None
def fillUID2(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    begin = row['bs']
    if (end - bg > 0.15):
        res = db[((db.SR==rd) & (db.bs == int((bg*10 + end*10)/2)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in ch_ids:
                ch_ids.append(x)
                return x
            return None
    else:
        return None
def fillUID3(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
    endARM = row['es']
    if (end - bg > 0.05) & (end - bg < 0.5):
        res = db[((db.SR==rd) & (db.es == endARM))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in ch_ids:
                ch_ids.append(x)
                return x
            return None
    return None
def fillUID4(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
#     endARM = row['es']
    if (end - bg > 0.25):
        res = db[((db.SR==rd) & (db.bs == int((bg*10 + end*10)/3)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in ch_ids:
                ch_ids.append(x)
                return x
            return None
    return None
def fillUID5(row):
    rd = row['SR']
    bg = row['BA']
    end = row['EA']
#     endARM = row['es']
    if (end - bg > 0.25):
        res = db[((db.SR==rd) & (db.es == int((bg*10 + end*10)/3)))].UID
        if list(res) == []:
            return None
        else:
            x = list(res)[0]
            if x not in ch_ids:
                ch_ids.append(x)
                return x
            return None
    return None

In [86]:
ch2.loc[:, 'UID'] = ch2.apply(fillUID, axis = 1)
ch_w2 = ch2[~np.isnan(ch2.UID)]
print len(ch_w2)
ch3 = ch2[np.isnan(ch2.UID)]
print len(ch3)

2382
17852


In [87]:
ch3.loc[:, 'UID'] = ch3.apply(fillUID2, axis = 1)
ch_w3 = ch3[~np.isnan(ch3.UID)]
print len(ch_w3)
ch4 = ch3[np.isnan(ch3.UID)]
print len(ch4)

7877
9975


In [88]:
ch4.loc[:, 'UID'] = ch4.apply(fillUID3, axis = 1)
ch_w4 = ch4[~np.isnan(ch4.UID)]
print len(ch_w4)
ch5 = ch4[np.isnan(ch4.UID)]
print len(ch5)

665
9310


In [89]:
ch5.loc[:, 'UID'] = ch5.apply(fillUID4, axis = 1)
ch_w5 = ch5[~np.isnan(ch5.UID)]
print len(ch_w5)
ch6 = ch5[np.isnan(ch5.UID)]
print len(ch6)

438
8872


In [90]:
ch6.loc[:, 'UID'] = ch6.apply(fillUID5, axis = 1)
ch_w6 = ch6[~np.isnan(ch6.UID)]
print len(ch_w6)
ch7 = ch6[np.isnan(ch6.UID)]
print len(ch7)

99
8773


In [91]:
print len(np.unique(ch_ids))
print len(db)
print len(ch)

16550
73046
28588


In [92]:
ch_all = pd.concat([ch_wid, ch_w2, ch_w3, ch_w4, ch_w5, ch_w6])
ch_all = ch_all.drop_duplicates(subset = ['UID'])

In [93]:
CH[['SR', 'Begin ARM', 'End ARM', u'Major Rehab Year']].head()

,SR,Begin ARM,End ARM,Major Rehab Year
0,2,3.96,4.50,1998
1,2,4.50,4.53,1982
2,2,4.53,4.80,1998
3,2,5.15,6.50,1998
4,2,6.50,6.66,1998


In [94]:
ch_final = pd.merge(ch_all, CH, how = 'left', left_on = ['SR', 'BA', 'EA'], right_on = ['SR', 'Begin ARM', 'End ARM'])
ch_FINAL = ch_final.dropna(subset = [u'Major Rehab Year'])
# CDS = cds_FINAL.drop_duplicates(subset = ['SR', 'Begin ARM', 'End ARM', 'Construction End Date'])

In [95]:
ch_FINAL[['SR', 'Begin ARM', 'End ARM','UID', u'Major Rehab Year']].head()

,SR,Begin ARM,End ARM,UID,Major Rehab Year
0,2,6.50,6.66,66,1998
1,2,11.32,11.49,114,1998
2,2,11.49,11.51,115,1998
3,2,13.69,13.72,137,1984
4,2,13.77,13.85,138,1984


In [96]:
ch_temp = CH.drop_duplicates(subset=['SR', 'Begin ARM', 'End ARM', 'Major Rehab Year'])
print len(ch_temp)

24690


In [97]:
print len(ch_FINAL)
print len(ch_ids)
print len(ch)

19253
16550
28588


In [98]:
ch_FINAL.to_csv('../FinalDatasets/ContractsHistory.csv')

In [99]:
ch_sorted = ch_FINAL.sort(columns = ['SR', 'Begin ARM', 'End ARM'])

In [100]:
ch_sorted.head()

,SR,BA,EA,bs,es,UID,RRT,RRQ,Begin ARM,End ARM,...,Analysis Period End Year,Future Total Value,Real Cost Per Lane Mile Year,Analysis Period ESALs,Analysis Period Vehicles,Sinking Fund Ratio,EUAC,EUAC Per Lane Mile,Dollars Per Mile Per ESAL,Dollars Per Mile Per Vehicle
284,2,0.00,0.10,0,1,1,NaN,NaN,0.00,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,2,0.13,0.28,1,2,2,FI,00389,0.13,0.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,2,0.29,0.32,2,3,3,Q1,29681,0.29,0.32,...,2055,NaN,87755.53,NaN,NaN,0.006550,12254.74,204245.67,NaN,NaN
302,2,0.39,0.48,3,4,4,CO,DIVISN,0.39,0.48,...,2014,NaN,20556.32,2275469.18,128186336,0.033582,9949.54,27637.61,0.9717,0.0172
301,2,0.58,0.63,5,6,6,CO,BROWNE,0.58,0.63,...,2014,NaN,26743.62,2167431.98,111591714,0.045820,4833.77,30211.06,0.7137,0.0139


In [101]:
su_final.head()

,SR,BA,EA,bs,es,UID
0,2,312.04,312.14,3120,3121,3121
1,2,171.15,171.25,1711,1712,1712
2,2,269.53,269.63,2695,2696,2696
3,2,22.12,22.23,221,222,222
4,2,115.81,115.91,1158,1159,1159


In [102]:
cds_all.head()

,SR,BA,EA,bs,es,UID
0,2,0.00,0.13,0,1,1
19,2,1.49,1.55,14,15,15
25,2,2.64,2.75,26,27,27
34,2,3.64,3.75,36,37,37
40,2,6.50,6.66,65,66,66


In [103]:
ch_all.head()

,SR,BA,EA,bs,es,UID
4,2,6.50,6.66,65,66,66
12,2,11.32,11.49,113,114,114
13,2,11.49,11.51,114,115,115
20,2,13.69,13.72,136,137,137
22,2,13.77,13.85,137,138,138


In [104]:
scc = pd.merge(su_final, cds_all, how = 'left', left_on = ['UID'], right_on = ['UID'])

In [106]:
scc2 = scc.dropna(subset = ['SR_y'])
print len(scc), len(scc2)

86850 31374


In [107]:
scc3 = pd.merge(scc2, ch_all, how = 'left', left_on = ['UID'], right_on = ['UID'])
scc3.head()

,SR_x,BA_x,EA_x,bs_x,es_x,UID,SR_y,BA_y,EA_y,bs_y,es_y,SR,BA,EA,bs,es
0,2,22.12,22.23,221,222,222,2,22.12,22.23,221,222,2,22.12,22.23,221,222
1,2,53.95,54.05,539,540,540,2,53.75,54.08,537,540,2,53.75,54.08,537,540
2,2,140.18,140.28,1401,1402,1402,2,206.99,213.54,2069,2135,NaN,NaN,NaN,NaN,NaN
3,2,15.48,15.58,154,155,155,2,15.45,15.48,154,154,2,15.45,15.48,154,154
4,2,113.21,113.31,1132,1133,1133,2,112.08,114.47,1120,1144,NaN,NaN,NaN,NaN,NaN


In [108]:
sccT = scc3.dropna(subset = ['SR'])
print len(sccT), len(scc3), len(scc2)

19200 31374 31374
